# mm3 Segmentation Debug Notebook

- Use this notebook to test your segmentation algorithm on your images, or write your own. 
- Then put your custom algorithm into mm3_helpers.py function segment_image

#### Load Modules

In [ ]:
#!/usr/bin/python
from __future__ import print_function

# import modules
import sys
import os
import time
import inspect
import yaml
import json # for importing tiff metdata
try:
    import cPickle as pickle # pickle
except:
    import pickle
import numpy as np
import scipy.signal as spsig # used in channel finding
import scipy.stats as spstats
from scipy.optimize import curve_fit
import struct # for interpretting strings as binary data
import re # regular expressions
import traceback
import copy
import warnings
import h5py

# Image analysis modules
from scipy import ndimage as ndi
from skimage import segmentation # used in make_masks and segmentation
from skimage.feature import match_template # used to align images
from skimage.filters import threshold_otsu
from skimage import morphology # many functions is segmentation used from this
from skimage.measure import regionprops # used for creating lineages

# Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 12}
mpl.rc('font', **font)
mpl.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
# you need to import mm3_helpers
sys.path.insert(0, '/path/to/mm3/')
import mm3_helpers as mm3
# import mm3_plots

### Load parameters file

- Put in the path to your parameters file!

In [ ]:
param_file_path = '../params.yaml'
params = mm3.init_mm3_helpers(param_file_path)

# # load channel_masks
# with open(os.path.join(params['ana_dir'], 'channel_masks.pkl'), 'r') as chnl_file:
#     channel_masks = pickle.load(chnl_file)

# load specs file
with open(os.path.join(params['ana_dir'], 'specs.yaml'), 'r') as specs_file:
    specs = yaml.safe_load(specs_file)

print(specs)

## Plot channel locations

In [ ]:
# fig = mm3_plots.channel_locations(channel_masks, filetype='channel_masks')
fig = mm3_plots.channel_locations(specs, filetype='specs')

# Segment image test

- Use this to test segmentation on one image

### Load some cells and images

- choose an FOV and Peak ID for channel that has cells in it.
- or let the code just choose the first peak with cells in it
- Same goes for a timepoint.

In [ ]:
# manually pick peak and timepoint to test
# fov_id = 1
# peak_id = 0

# automatically choose first analyzed peak
for fov_id, peaks in specs.items():
    for peak_id, spec in peaks.items():
        if spec == 1:
            # just break out with the current fov_id and peak_id
            break
            
timepoint = 0
    
print('FOV %d peak %d time point %d' % (fov_id, peak_id, timepoint))

# load phase contrast image
img_stack = mm3.load_stack(fov_id, peak_id, color=params['phase_plane'])
pc_img = img_stack[timepoint]

# load subtracted image
sub_stack = mm3.load_stack(fov_id, peak_id, color='sub_' + params['phase_plane'])
# take first time point
sub_img = sub_stack[timepoint]

# plot
fig, axes = plt.subplots(ncols=2, sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(pc_img, cmap=plt.cm.gray)
ax[1].imshow(sub_img, cmap=plt.cm.gray)

for axis in axes:
    axis.get_xaxis().set_visible(False)
    axis.get_yaxis().set_visible(False)
    
plt.show()

### Choose parameters and run segmentation
- This segmentation script is the same one in mm3.segment_image function

In [ ]:
# choose parameters 
OTSU_threshold = 1.0
first_opening_size = 2
distance_threshold = 2
second_opening_size = 1
min_object_size = 25

In [ ]:
# threshold image
thresh = threshold_otsu(sub_img) # finds optimal OTSU thershhold value
threshholded = sub_img > OTSU_threshold*thresh # will create binary image

# if there are no cells, good to clear the border
# because otherwise the OTSU is just for random bullshit, most
# likely on the side of the image
threshholded = segmentation.clear_border(threshholded)
thresholded_plot = threshholded.copy()

# Opening = erosion then dialation.
# opening smooths images, breaks isthmuses, and eliminates protrusions.
# "opens" dark gaps between bright features.
morph = morphology.binary_opening(threshholded, morphology.disk(first_opening_size))
opening_plot = morph.copy()

# if this image is empty at this point (likely if there were no cells), just return
# the morphed image which is a zero array
if np.amax(morph) == 0:
    print('No cells in after thresholding and opening')
    pass

### Calculate distnace matrix, use as markers for random walker (diffusion watershed)
# Generate the markers based on distance to the background
distance = ndi.distance_transform_edt(morph)
distance_plot = distance.copy()

# threshold distance image
distance_thresh = np.zeros_like(distance)
distance_thresh[distance < distance_threshold] = 0
distance_thresh[distance >= distance_threshold] = 1
distance_thresh_plot = distance_thresh.copy()

# do an extra opening on the distance
distance_opened = morphology.binary_opening(distance_thresh, 
                                            morphology.disk(second_opening_size))
distance_opened_plot = distance_opened.copy()

# remove artifacts connected to image border
cleared = segmentation.clear_border(distance_opened)
# remove small objects. Remove small objects wants a
# labeled image and will fail if there is only one label. Return zero image in that case
# could have used try/except but remove_small_objects loves to issue warnings.
cleared, label_num = morphology.label(cleared, connectivity=1, return_num=True)
if label_num > 1:
    cleared = morphology.remove_small_objects(cleared, min_size=min_object_size)
else:
    pass

# relabel now that small objects and labels on edges have been cleared
markers = morphology.label(cleared)
markers_plot = markers.copy()

# label using the random walker (diffusion watershed) algorithm
# the binary image for the watershed, which uses the unmodified OTSU threshold
threshholded_watershed = sub_img > thresh
threshholded_watershed = segmentation.clear_border(threshholded_watershed)

# set anything outside of OTSU threshold to -1 so it will not be labeled
markers[threshholded_watershed == 0] = -1
# here is the main algorithm
labeled_image = segmentation.random_walker(-1*sub_img, markers)
# put negative values back to zero for proper image
labeled_image[labeled_image == -1] = 0

# or watershed
# labeled_image = morphology.watershed(-1*sub_img, markers, mask=threshholded)

### Plot it up. 
fig, axes = plt.subplots(ncols=10, figsize=(16, 10), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].set_title('image')
ax[0].imshow(pc_img, cmap=plt.cm.gray)

ax[1].set_title('subtracted')
ax[1].imshow(sub_img, cmap=plt.cm.gray)

ax[2].set_title('OTSU')
ax[2].imshow(thresholded_plot, cmap=plt.cm.gray)

ax[3].set_title('Opened')
ax[3].imshow(opening_plot, cmap=plt.cm.gray)

ax[4].set_title('Distance')
ax[4].imshow(distance_plot, cmap=plt.cm.gray)

ax[5].set_title('Dist. Tresh')
ax[5].imshow(distance_thresh_plot, cmap=plt.cm.gray)

ax[6].set_title('Dist. Opened')
ax[6].imshow(distance_opened_plot, cmap=plt.cm.gray)

ax[7].set_title('Markers')
ax[7].imshow(markers_plot)

ax[8].set_title('Diffusion')
ax[8].imshow(labeled_image)

ax[9].set_title('Overlay')
ax[9].imshow(sub_img, cmap=plt.cm.gray)
ax[9].imshow(labeled_image, alpha=0.5)

for axis in axes:
    axis.get_xaxis().set_visible(False)
    axis.get_yaxis().set_visible(False)

# Segmentation parameter sweep
- Here, you can plot the above graph for combinations of parameters for many time points
- First, we make a directory for these images
- Then, we define a segmentation function which is the same as the one above and in mm3. 
- Finally, we choose a range of parameters and the desired number of samples

In [ ]:
# Make directory for this 
seg_test_dir = os.path.join(params['experiment_directory'], params['analysis_directory'],
                       'segment_testing')
if not os.path.exists(seg_test_dir):
    os.makedirs(seg_test_dir)

In [ ]:
def segment(fov_id, peak_id, timepoint, 
            OTSU_threshold, first_opening_size, distance_threshold, second_opening_size):

    ### Load images ##############################################
    # load phase contrast image
    img_stack = mm3.load_stack(fov_id, peak_id, color=params['phase_plane'])
    pc_img = img_stack[timepoint]

    # load subtracted image
    sub_stack = mm3.load_stack(fov_id, peak_id, color='sub_' + params['phase_plane'])
    # take first time point
    sub_img = sub_stack[timepoint]
    
    ### Do segmentation ##########################################
    # threshold image
    thresh = threshold_otsu(sub_img) # finds optimal OTSU thershhold value
    threshholded = sub_img > OTSU_threshold*thresh # will create binary image

    # if there are no cells, good to clear the border
    # because otherwise the OTSU is just for random bullshit, most
    # likely on the side of the image
    threshholded = segmentation.clear_border(threshholded)
    thresholded_plot = threshholded.copy()

    # Opening = erosion then dialation.
    # opening smooths images, breaks isthmuses, and eliminates protrusions.
    # "opens" dark gaps between bright features.
    morph = morphology.binary_opening(threshholded, morphology.disk(first_opening_size))
    opening_plot = morph.copy()

    # if this image is empty at this point (likely if there were no cells), just return
    # the morphed image which is a zero array
    if np.amax(morph) == 0:
        print('No cells in after thresholding and opening')
        pass

    ### Calculate distnace matrix, use as markers for random walker (diffusion watershed)
    # Generate the markers based on distance to the background
    distance = ndi.distance_transform_edt(morph)
    distance_plot = distance.copy()

    # threshold distance image
    distance_thresh = np.zeros_like(distance)
    distance_thresh[distance < distance_threshold] = 0
    distance_thresh[distance >= distance_threshold] = 1
    distance_thresh_plot = distance_thresh.copy()

    # do an extra opening on the distance
    distance_opened = morphology.binary_opening(distance_thresh, 
                                                morphology.disk(second_opening_size))
    distance_opened_plot = distance_opened.copy()

    # remove artifacts connected to image border
    cleared = segmentation.clear_border(distance_opened)
    # remove small objects. Remove small objects wants a
    # labeled image and will fail if there is only one label. Return zero image in that case
    # could have used try/except but remove_small_objects loves to issue warnings.
    cleared, label_num = morphology.label(cleared, connectivity=1, return_num=True)
    if label_num > 1:
        cleared = morphology.remove_small_objects(cleared, min_size=min_object_size)
    else:
        pass

    # relabel now that small objects and labels on edges have been cleared
    markers = morphology.label(cleared)
    markers_plot = markers.copy()

    # label using the random walker (diffusion watershed) algorithm
    # the binary image for the watershed, which uses the unmodified OTSU threshold
    threshholded_watershed = sub_img > thresh
    threshholded_watershed = segmentation.clear_border(threshholded_watershed)

    # set anything outside of OTSU threshold to -1 so it will not be labeled
    markers[threshholded_watershed == 0] = -1
    # here is the main algorithm
    labeled_image = segmentation.random_walker(-1*sub_img, markers)
    # put negative values back to zero for proper image
    labeled_image[labeled_image == -1] = 0

    # or watershed
    # labeled_image = morphology.watershed(-1*sub_img, markers, mask=threshholded)

    ### Plot it up. 
    fig, axes = plt.subplots(ncols=10, figsize=(16, 10), sharex=True, sharey=True)
    ax = axes.ravel()

    ax[0].set_title('image')
    ax[0].imshow(pc_img, cmap=plt.cm.gray)

    ax[1].set_title('subtracted')
    ax[1].imshow(sub_img, cmap=plt.cm.gray)

    ax[2].set_title('OTSU')
    ax[2].imshow(thresholded_plot, cmap=plt.cm.gray)

    ax[3].set_title('Opened')
    ax[3].imshow(opening_plot, cmap=plt.cm.gray)

    ax[4].set_title('Distance')
    ax[4].imshow(distance_plot, cmap=plt.cm.gray)

    ax[5].set_title('Dist. Tresh')
    ax[5].imshow(distance_thresh_plot, cmap=plt.cm.gray)

    ax[6].set_title('Dist. Opened')
    ax[6].imshow(distance_opened_plot, cmap=plt.cm.gray)

    ax[7].set_title('Markers')
    ax[7].imshow(markers_plot)

    ax[8].set_title('Diffusion')
    ax[8].imshow(labeled_image)

    ax[9].set_title('Overlay')
    ax[9].imshow(sub_img, cmap=plt.cm.gray)
    ax[9].imshow(labeled_image, alpha=0.5)

    for axis in axes:
        axis.get_xaxis().set_visible(False)
        axis.get_yaxis().set_visible(False)
    
    ax[0].get_yaxis().set_visible(True)
    ax[0].set_ylabel('FOV {:03d}, Peak {:04d}, t {:04d}'.format(fov_id, peak_id, timepoint), size=16)

    fig.suptitle('OTSU = {}, Opening 1 = {}, Distance Threshold = {}, Opening 2 = {}'.format(otsu_thresh, osize1, dthresh, osize2), size=20)
        
    return fig, ax

In [ ]:
# choose parameter ranges
OTSU_thresholds = [1.0]
first_opening_sizes = [2, 3]
distance_thresholds = [2]
second_opening_sizes = [1]
min_object_size = 20

# create list of all combinations 
p_combos = []
for otsu_thresh in OTSU_thresholds:
    for osize1 in first_opening_sizes:
        for dthresh in distance_thresholds:
            for osize2 in second_opening_sizes:
                p_combos.append((otsu_thresh, osize1, dthresh, osize2))
            
# or simply choose what you want the parameters to be
# p_combos = [(1.2,3,2,2), (1.4,3,2,1)]

# determine number of images to be analyzed 
no_samples = 25
# list contains tuples of (fov_id, peak_id, timepoint)
sample_ids = [] 
timepoints = None # place holder for number of time points
for fov_id in specs.keys():
    for peak_id, spec in specs[fov_id].items():
        if spec == 1:
            
            # If we don't know the number of time points, figure that out now
            if timepoints is None:
                # should pull out the number of timepoints. 
                timepoints = len(mm3.load_stack(fov_id, peak_id, color='c1'))
            
            # now put the fov, peak, and a random timepoint in a tuple
            sample_ids.append((fov_id, peak_id, np.random.choice(timepoints, 1)[0]))
            
# choose a random subset of these and sort them
sample_indicies = np.random.choice(len(sample_ids), no_samples)
sample_ids = sorted([sample_ids[i] for i in sample_indicies]) 

# cycle through all sample and parameter combinations
for sample in sample_ids:
    fov_id, peak_id, timepoint = sample
    for p_combo in p_combos:
        otsu_thresh, osize1, dthresh, osize2 = p_combo
        try:
            fig, ax = segment(fov_id, peak_id, timepoint, 
                              otsu_thresh, osize1, dthresh, osize2)
            
        except:
            print('{}-{}-{}-{}_xy{:03d}_p{:04d}_t{:04d} has no cells'.format(otsu_thresh, osize1, dthresh, osize2, 
                                                                 fov_id, peak_id, timepoint))
            continue
        seg_filename = '{}-{}-{}-{}_xy{:03d}_p{:04d}_t{:04d}.png'.format(otsu_thresh, osize1, dthresh, osize2, 
                                                                 fov_id, peak_id, timepoint)
        seg_filepath = os.path.join(seg_test_dir, seg_filename)
        fig.savefig(seg_filepath, dpi=75)
        plt.close()

# Plot lineages
- This create the graph of the lineages super-imposed on the segmented images. 
- This is to be done after segmentation for an entire channel

In [ ]:
# The linage plotting function is in mm3.plots
import mm3_plots as mm3_plots

In [ ]:
# plotting lineage trees for complete cells
with open(os.path.join(params['cell_dir'], 'all_cells.pkl'), 'r') as cell_file:
    Cells = pickle.load(cell_file)
with open(os.path.join(params['cell_dir'], 'complete_cells.pkl'), 'r') as cell_file:
    Cells2 = pickle.load(cell_file)
    Cells2 = mm3_plots.find_cells_of_birth_label(Cells2, label_num=[1,2])

lin_dir = os.path.join(params['experiment_directory'], params['analysis_directory'],
                       'lineages')
if not os.path.exists(lin_dir):
    os.makedirs(lin_dir)

In [ ]:
# determine number of lineages to make
no_samples = 10
# list contains tuples of (fov_id, peak_id, timepoint)
sample_ids = [] 
for fov_id in specs.keys():
    for peak_id, spec in specs[fov_id].items():
        if spec == 1:
            # now put the fov, peak, and a random timepoint in a tuple
            sample_ids.append((fov_id, peak_id))
            
# choose a random subset of these and sort them
sample_indicies = np.random.choice(len(sample_ids), no_samples)
sample_ids = sorted([sample_ids[i] for i in sample_indicies]) 

In [ ]:
for sample in sample_ids:
    fov_id, peak_id = sample
    fig, ax = mm3_plots.plot_lineage_images(Cells, fov_id, peak_id, Cells2,
                                            color='sub_' + params['phase_plane'])
    lin_filename = params['experiment_name'] + '_xy%03d_p%04d_lin.png' % (fov_id, peak_id)
    lin_filepath = os.path.join(lin_dir, lin_filename)
    fig.savefig(lin_filepath, dpi=75)
    plt.close(fig)